In [ ]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
from feeds import BackFeed,DataFeed
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed
from featfuncs import add_global_indices_feed

In [ ]:
from feed_env import Episode

In [ ]:
from ruleagents import AdaMomCMOAgent, GapBetAgent

In [ ]:
from rulestrats import AdaMomCMOADF, GapBet

In [ ]:
import aspectlib

In [ ]:
def do_nothing(dfD):
    empty={t:0 for t in dfD}
    return empty,empty,empty
def always_buy(dfD):
    buy={t:1 for t in dfD}
    empty={t:0 for t in dfD}
    return buy,empty,empty
def always_sell(dfD):
    sell={t:-1 for t in dfD}
    empty={t:0 for t in dfD}
    return sell,empty,empty

In [ ]:
def get_alt_data():
    global feed
    aD={'gdata':feed.gdata}
    return aD

Strategy Development: ADF Momentum

In [ ]:
from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate

In [ ]:
data=pd.read_csv('./capvol100.csv')

In [ ]:
tickers=list(data.iloc[0:10]['ticker'].values)

In [ ]:
feed=BackFeed(tickers=tickers,nd=3,nw=1,interval='5m')

In [ ]:
add_addl_features_feed(feed,tickers=feed.tickers)

In [ ]:
add_sym_feature_feed(feed,tickers=feed.tickers)

In [ ]:
add_global_indices_feed(feed)

In [ ]:
import pickle

In [ ]:
# with open('../../temp_data/btfeed.pickle','wb') as f: pickle.dump(feed,f)

In [ ]:
with open('../../temp_data/btfeed.pickle','rb') as f: feed=pickle.load(f)

In [ ]:
gapagent=GapBetAgent(direction=-1)

In [ ]:
gapagent=GapBetAgent(direction=-1)
gapagent.set_alt_data(alt_data_func=get_alt_data)

In [ ]:
gapstrat=GapBet()
gapstrat.set_alt_data(alt_data_func=get_alt_data)

In [ ]:
adamom=AdaMomCMOAgent()
adamom.use_memory=True

In [ ]:
adamom.debug=True

In [ ]:
adastrat=AdaMomCMOADF()

In [ ]:
import importlib
import sys,os

In [ ]:
importlib.reload(sys.modules['feed_env'])
from feed_env import Episode

In [ ]:
importlib.reload(sys.modules['backtest'])
from backtest import Backtest

In [ ]:
importlib.reload(sys.modules['rulestrats'])
from rulestrats import AdaMomCMOADF,GapBet

In [ ]:
# def save_func(df): return(df.iloc[-1]['CMO_14'])

In [ ]:
@aspectlib.Aspect
def my_decorator(*args, **kwargs):
    # print("Got called with args: %s kwargs: %s" % (args, kwargs))
    # result = yield
    # print(" ... and the result is: %s" % (result,))
    state,rew,done,exit_type = yield
    args[0].policy.reward((rew,exit_type))
    return state,rew,done,exit_type

In [ ]:
aspectlib.weave(Episode, my_decorator, methods='env_step')

In [ ]:
bt=Backtest(feed,tickers=feed.tickers,add_features=False,target=.5,stop=.5,txcost=0.001,
            loc_exit=True,scan=True,topk=3,deploy=True,save_dfs=False,
            save_func=None)

In [ ]:
bt.run_all(tickers=feed.tickers,model=adamom,verbose=False)

In [ ]:
bt.results

In [ ]:
adamom.logL

In [ ]:
len(L)

In [ ]:
M=adamom.memory.perceptual_memory

In [ ]:
M[0]

In [ ]:
[(M[t]['world_state'][1].keys(),M[t]['ticker']) for t in M][0:200]

In [ ]:
from ruleagents import COLS

In [ ]:
A=[(M[t]['world_state'],M[t]['info']) for t in M]

In [ ]:
[L[t] for t in range(len(L)) if len(L[t])>2]

In [ ]:
P=[percept_to_state(L[t]) for t in range(len(L))]

In [ ]:
[torch.tensor(M[t]['default']['percept'][2][COLS].values) for t in M if M[t]['default']['percept'][0]=='exit'][0:200]

In [ ]:
[torch.tensor(M[t]['default']['percept'][1]
              [M[t]['default']['info']['ticker']][COLS].values) for t in M if M[t]['default']['percept'][0]=='entry'][0:200]

In [ ]:
[(t,M[t]['default'].keys()) for t in M if M[t]['default']['percept'][0]=='exit']

In [ ]:
rew=lambda x: x['reward'] if 'reward' in x else None

In [ ]:
[(t,M[t]['ticker'],M[t]['world_state'][0],
 M[t]['action'],rew(M[t])) for t in M][0:500]

In [ ]:
S=adamom.memory.sar_memory

In [ ]:
S[5]

In [ ]:
act=lambda x: x['action'] if 'action' in x else 'no action'

In [ ]:
[(t,act(S[t]),S[t]['reward']) for t in S if t>=0][0:500]

In [ ]:
bt.results

In [ ]:
bt.results

In [ ]:
from datetime import datetime

In [ ]:
datetime.today()

In [ ]:
bt.returns

In [ ]:
M=adamom.memory.sar_memory

In [ ]:
[M[t]['reward'] for t in M if 'reward' in M[t]]

Host Service via Anvil

TBD: enable remote calling for agent strategies - add reward function here and call in tradeserver

In [ ]:
import anvil.server

In [ ]:
# DeepTrader:
anvil.server.connect("STKYCIXUMGFV2CURGMCMOFUV-MBK5ZXFC4MCKYKOE")

In [ ]:
momStrat=MomStrat()

In [ ]:
momStrat=AdaMomCMOADF()

In [ ]:
@anvil.server.callable
def check_entry_batch_mom2(dfD):
    global momStrat
    dfDloc={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return MomStrat.Check(momStrat,dfDloc)

In [ ]:
@anvil.server.callable
def check_exit_batch_mom2(dfD,posf):
    global momStrat
    posloc=pd.DataFrame(posf)
    dfDloc={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return MomStrat.Exit(momStrat,dfDloc,posloc)